In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from neuroCombat import neuroCombat

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har

In [ ]:
# Datasets for this work
EDIS_path = '../our_datasets/EDIS/'
HELIUS_path = '../our_datasets/HELIUS/'
Insight46_path = '../our_datasets/Insight46/'
HELIUS_path = '../our_datasets/HELIUS/'
SABRE_path = '../our_datasets/SABRE/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)
Insight46_file = os.path.join(Insight46_path, file_name)
HELIUS_file = os.path.join(HELIUS_path, file_name)

TOP = pd.read_csv(TOP_file, index_col=0 )
MRI = pd.read_csv(MRI_file, index_col=0)
HELIUS = pd.read_csv(HELIUS_file, index_col=0) # 
EDIS = pd.read_csv(EDIS_file, index_col=0 )
SABRE = pd.read_csv(SABRE_file, index_col=0)
INSI = pd.read_csv(Insight46_file, index_col=0) # 

In [ ]:
HELIUS[HELIUS['participant_id']=='sub-153852_1']

In [ ]:
SABRE[SABRE['participant_id']=='sub-153852_1']

In [ ]:
HELIUS.loc[HELIUS['participant_id']=='sub-153852_1', 'participant_id'] = 'sub-153852_1H'

In [ ]:
HELIUS.Sex.unique()

In [ ]:
HELIUS.head(3)

In [ ]:
sex_mapping = {1:0,2:1}

EDIS = EDIS.assign(Sex = EDIS.Sex.map(sex_mapping))
EDIS.head(3)

In [ ]:
INSI = INSI.assign(Sex = INSI.Sex.map(sex_mapping))
INSI.head(3)

In [ ]:
SABRE = SABRE.assign(Sex = SABRE.Sex.map(sex_mapping))
SABRE.head(3)

In [ ]:
TOP = TOP.drop(['Site', 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['Site', 'ID'],axis = 1)
MRI.tail(3)

In [ ]:
TOPMRI = pd.concat([MRI ,TOP])
TOPMRI.head(3)

In [ ]:
HELIUS = HELIUS.drop(['Site', 'ID'],axis = 1)
HELIUS.tail(3)

In [ ]:
TOPMRI.shape